In [7]:
HOME = '/home/nguyen-duc-thien/Tours/AI'
%cd {HOME}

/home/nguyen-duc-thien/Tours/AI


/home/nguyen-duc-thien/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import math
import pickle
from collections import Counter, defaultdict
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /home/nguyen-duc-
[nltk_data]     thien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nguyen-duc-
[nltk_data]     thien/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/nguyen-duc-
[nltk_data]     thien/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
np.random.seed(42)

In [10]:
dataset = pd.read_csv("/home/nguyen-duc-thien/Tours/AI/output.csv")
dataset = dataset[:500]
print(dataset.head())

                                              Review  Rating     Label
0  nice hotel expensive parking got good deal sta...       4  positive
1  ok nothing special charge diamond member hilto...       2  negative
2  nice rooms not 4* experience hotel monaco seat...       3  positive
3  unique, great stay, wonderful time hotel monac...       5  positive
4  great stay great stay, went seahawk game aweso...       5  positive


dataset = pd.read_csv("/kaggle/input/hotels/output.csv")
dataset = dataset[:100]
print(dataset.head())

In [11]:
print("NaN values in the dataset:")
print(dataset.isnull().sum())

# Remove rows with NaN values or fill them
dataset = dataset.dropna()

def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
dataset['Review'] = dataset['Review'].apply(remove_tags)

def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
dataset['Review'] = dataset['Review'].apply(special_char)

def convert_lower(text):
   return text.lower()
dataset['Review'] = dataset['Review'].apply(convert_lower)
dataset['Review'][1]

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return [x for x in words if x not in stop_words]
dataset['Review'] = dataset['Review'].apply(remove_stopwords)
dataset['Review'][1]

def lemmatize_word(text):
  wordnet = WordNetLemmatizer()
  return " ".join([wordnet.lemmatize(word) for word in text])
dataset['Review'] = dataset['Review'].apply(lemmatize_word)
dataset['Review'][1]

NaN values in the dataset:
Review    0
Rating    0
Label     0
dtype: int64


'ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle start booked suite paid extra website description suite bedroom bathroom standard hotel room took printed reservation desk showed said thing like tv couch ect desk clerk told oh mixed suite description kimpton website sorry free breakfast got kidding embassy suit sitting room bathroom bedroom unlike kimpton call suite 5 day stay offer correct false advertising send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty reply solution send email trip guest survey follow email mail guess tell concerned guest staff ranged indifferent helpful asked desk good breakfast spot neighborhood hood told hotel gee best breakfast spot seattle 1 2 block away convenient hotel know exist arrived late night 11 pm inside run bellman busy chating cell phone help bag prior arrival emailed hotel 

In [12]:
# Training data and labels
training_texts = dataset['Review']
training_labels = dataset['Label']

print(len(dataset))
print(dataset.head()['Review'])

500
0    nice hotel expensive parking got good deal sta...
1    ok nothing special charge diamond member hilto...
2    nice room 4 experience hotel monaco seattle go...
3    unique great stay wonderful time hotel monaco ...
4    great stay great stay went seahawk game awesom...
Name: Review, dtype: object


In [13]:
def save_model(classifier, filename):
    with open(filename, 'wb') as file:
        pickle.dump(classifier, file)

def load_model(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

## Text Classifier

In [14]:
class TextClassifier:
    def __init__(self):
        self.training_data = []
        self.vocabulary = set()
        self.idf_weights = {}

    def _tokenize(self, text):
        return text.lower().split()

    def _build_vocabulary(self, texts):
        self.vocabulary = set()
        for text in texts:
            self.vocabulary.update(self._tokenize(text))

    def _compute_tfidf_weights(self, texts):
        # Count document frequencies
        doc_freq = defaultdict(int)
        total_docs = len(texts)

        for text in texts:
            unique_terms = set(self._tokenize(text))
            for term in unique_terms:
                doc_freq[term] += 1

        # Compute IDF weights
        self.idf_weights = {
            term: math.log(total_docs / (count + 1))
            for term, count in doc_freq.items()
        }

    def _text_to_vector(self, text):
        tokens = self._tokenize(text)
        vector = [0] * len(self.vocabulary)

        # Count term frequencies
        token_counts = Counter(tokens)

        for i, term in enumerate(self.vocabulary):
            # TF-IDF calculation
            tf = token_counts.get(term, 0) / len(tokens)
            idf = self.idf_weights.get(term, 0)
            vector[i] = tf * idf

        return vector

## KNN Classifier

In [15]:
class KNNClassifier(TextClassifier):
    def __init__(self, k=5):
        super().__init__()
        self.k = k
        self.training_vectors = None
        self.training_labels = None

    def _cosine_similarity(self, vec1, vec2):
        dot_product = sum(x*y for x, y in zip(vec1, vec2))
        magnitude1 = math.sqrt(sum(x*x for x in vec1))
        magnitude2 = math.sqrt(sum(x*x for x in vec2))

        return dot_product / (magnitude1 * magnitude2 + 1e-10)

    # def train(self, training_texts, labels):
    #     # Build vocabulary and compute IDF weights
    #     self._build_vocabulary(training_texts)
    #     self._compute_tfidf_weights(training_texts)

    #     # Compute vector representations
    #     self.training_data = [
    #         (self._text_to_vector(text), label)
    #         for text, label in zip(training_texts, labels)
    #     ]

    def train(self, training_texts, labels):
        # Build vocabulary and compute IDF weights
        self._build_vocabulary(training_texts)
        self._compute_tfidf_weights(training_texts)

        # Convert to numpy arrays for faster computations
        self.training_vectors = np.array([
            self._text_to_vector(text) for text in training_texts
        ])
        self.training_labels = np.array(labels)

    # def predict(self, new_text):
    #     new_vector = self._text_to_vector(new_text)
    #     point_non_zero = [point for point in new_vector if point != 0.0]
    #     print(point_non_zero)

    #     # Compute similarities to training vectors
    #     similarities = [
    #         (self._cosine_similarity(new_vector, train_vector), label)
    #         for train_vector, label in self.training_data
    #     ]

    #     # Sort and get k nearest neighbors
    #     similarities.sort(reverse=True, key=lambda x: x[0])
    #     print(similarities)
    #     k_nearest = similarities[:self.k]

    #     # Majority voting
    #     label_counts = Counter(label for _, label in k_nearest)
    #     return label_counts.most_common(1)[0][0]

    def predict(self, new_text):
        # Convert new text to vector using numpy
        new_vector = np.array(self._text_to_vector(new_text))

        # Create a dictionary to group vectors by label
        label_to_vectors = defaultdict(list)
        for vector, label in zip(self.training_vectors, self.training_labels):
            label_to_vectors[label].append(vector)

        # Random 50 vectors for each class/label
        random_train_vectors = []
        for label, vectors in label_to_vectors.items():
            selected_vectors = []
            vectors = np.array(vectors)
            if len(vectors) > 50:
                selected_indices = np.random.choice(len(vectors), size=50, replace=False)
                selected_vectors = vectors[selected_indices]
            else:
                selected_vectors = vectors
            random_train_vectors.extend(selected_vectors)
            print(label, len(selected_vectors))
        # Compute similarities using vectorized operations
        similarities = np.array([
            self._cosine_similarity(new_vector, train_vector)
            for train_vector in np.array(random_train_vectors)
        ])

        # Get indices of k nearest neighbors
        k_nearest_indices = np.argpartition(similarities, -self.k)[-self.k:]

        # Select labels for k nearest neighbors
        k_nearest_labels = self.training_labels[k_nearest_indices]

        # Majority voting
        label_counts = Counter(k_nearest_labels)
        return label_counts.most_common(1)[0][0]

In [16]:
# Create and train classifier
knn_classifier = KNNClassifier(k=9)
knn_classifier.train(training_texts, training_labels)

# Save model
save_model(knn_classifier, 'knn_classifier_model.pkl')

## Naive Bayes Classifier

In [17]:
class NaiveBayesClassifier(TextClassifier):
    def __init__(self):
        super().__init__()

    def train(self, texts, labels):
        # Compute class probabilities
        class_counts = Counter(labels)
        self.class_probs = {
            cls: count/len(labels) for cls, count in class_counts.items()
        }

        # Compute IDF weights
        self._compute_tfidf_weights(texts)

        # Term frequencies per class
        self.term_freq = {}
        for cls in set(labels):
            self.term_freq[cls] = defaultdict(int)

        self.class_total_terms = defaultdict(int)

        # Process each document
        for text, cls in zip(texts, labels):
            terms = self._tokenize(text)
            for term in terms:
                self.term_freq[cls][term] += 1
                self.class_total_terms[cls] += 1

    def predict(self, text):
        terms = self._tokenize(text)
        class_scores = {}

        # Compute score for each class
        for cls in self.class_probs:
            # Start with log of class probability
            score = math.log(self.class_probs[cls])

            # Add log probabilities of terms
            for term in terms:
                # Laplace smoothing
                term_count = self.term_freq[cls][term]
                total_terms = self.class_total_terms[cls]

                # Weight term by IDF
                idf_weight = self.idf_weights.get(term, 0)

                # Compute log probability with smoothing
                term_prob = (term_count + 1) / (total_terms + len(self.idf_weights))
                score += math.log(term_prob) * idf_weight

            class_scores[cls] = score

        # Return class with highest score
        print(class_scores)
        return max(class_scores, key=class_scores.get)

In [18]:
# Create and train classifier
naive_bayes_classifier = NaiveBayesClassifier()
naive_bayes_classifier.train(training_texts, training_labels)

# Save model
save_model(naive_bayes_classifier, 'bayes_classifier_model.pkl')

## Decision Tree Classifier

In [19]:
# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_texts)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, training_labels, test_size=0.1, random_state=42)

# Train the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)
decision_tree_classifier.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = decision_tree_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.64      0.69      0.67        13
    positive       0.89      0.86      0.88        37

    accuracy                           0.82        50
   macro avg       0.77      0.78      0.77        50
weighted avg       0.82      0.82      0.82        50



In [20]:
# Save vectorizer
save_model(vectorizer, 'vectorizer.pkl')

# Save model
save_model(decision_tree_classifier, 'tree_classifier_model.pkl')

In [21]:
from sklearn.tree import export_text

# Export tree structure as text
tree_rules = export_text(decision_tree_classifier, feature_names=vectorizer.get_feature_names_out())
print(tree_rules)

|--- great <= 0.02
|   |--- location <= 0.04
|   |   |--- comfortable <= 0.03
|   |   |   |--- pike <= 0.02
|   |   |   |   |--- large <= 0.07
|   |   |   |   |   |--- wonderful <= 0.01
|   |   |   |   |   |   |--- walking <= 0.09
|   |   |   |   |   |   |   |--- parking <= 0.10
|   |   |   |   |   |   |   |   |--- excellent <= 0.05
|   |   |   |   |   |   |   |   |   |--- posh <= 0.11
|   |   |   |   |   |   |   |   |   |   |--- kind <= 0.06
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 9
|   |   |   |   |   |   |   |   |   |   |--- kind >  0.06
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |--- posh >  0.11
|   |   |   |   |   |   |   |   |   |   |--- class: positive
|   |   |   |   |   |   |   |   |--- excellent >  0.05
|   |   |   |   |   |   |   |   |   |--- class: positive
|   |   |   |   |   |   |   |--- parking >  0.10
|   |   |   |   |   |   |   |   |--- class: positive
|   |   |   

In [22]:
def preprocess_text(text):
    """Comprehensive text preprocessing function"""
    # Remove tags
    text = re.sub('<.*?>', '', str(text))

    # Remove special characters
    text = ''.join([x if x.isalnum() else ' ' for x in text])

    # Convert to lowercase
    text = text.lower()

    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize
    wordnet = WordNetLemmatizer()
    words = [wordnet.lemmatize(word) for word in words]

    return ' '.join(words)

In [23]:
def predict_text(text):
    # Load the saved model
    decision_tree_classifier = load_model('tree_classifier_model.pkl')
    vectorizer = load_model('vectorizer.pkl')

    # Preprocess the input text
    processed_text = preprocess_text(text)

    # Vectorize using the same vectorizer
    vector = vectorizer.transform([processed_text])

    # Predict
    prediction = decision_tree_classifier.predict(vector)

    return prediction[0]

## Load models

In [24]:
knn_model = load_model('knn_classifier_model.pkl')
naive_bayes_model = load_model('bayes_classifier_model.pkl')
decision_tree_model = load_model('tree_classifier_model.pkl')

## Predictions

In [25]:
# Test predictions
test_texts = ['''Thien is the best ugly boi in the the gioi''']

for text in test_texts:
    knn_prediction = knn_model.predict(text)
    nb_prediction = naive_bayes_model.predict(text)
    dt_prediction = predict_text(text)
    print(f"KNN Predicted Label: {knn_prediction}")
    print(f"Naive Bayes Predicted Label: {nb_prediction}")
    print(f"Decision Trees Predicted Label: {dt_prediction}")

positive 50
negative 50


{'positive': -64.80720209968936, 'negative': -56.770639781379366}
KNN Predicted Label: positive
Naive Bayes Predicted Label: negative
Decision Trees Predicted Label: negative


In [27]:
!pip install flask flask_cors pyngrok

from IPython.display import clear_output
clear_output()

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS, cross_origin
from pyngrok import ngrok

ngrok.set_auth_token("2pzbwE4yIbzv7a2zPeAddecREOJ_yiihsHcPQFezq7RxsSLL")

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@app.route('/', methods=['GET'])
@cross_origin()
def root():
  return "Hotels review"
# Model API
@app.route('/api/v1/reviews', methods=['POST'])
@cross_origin()
def classify():
    # Message format
    # {
    #     "comments": "article text to classify",
    #     "model": "knn/bayes/tree",
    # }

    data = request.get_json()

    if "comments" not in data:
        return jsonify({
            "error": "Please provide a comments to classify"
        })

    userQuery = data["comments"]
    model = data.get("model", "bayes")

    prediction = None
    if model == "knn":
        prediction = knn_model.predict(userQuery)
    elif model == "bayes":
        prediction = naive_bayes_model.predict(userQuery)
    elif model == "tree":
        prediction = predict_text(userQuery)
    else:
        return jsonify({
            "error": "Invalid model"
        })

    return jsonify({
        "result": prediction
    })

url = ngrok.connect(8000).public_url
print('Global URL:', url)

if __name__ == '__main__':
    app.run(debug=False, port=8000)

Global URL: https://6e7c-58-186-78-230.ngrok-free.app                                               
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit


positive 50
negative 50


127.0.0.1 - - [10/Dec/2024 02:49:10] "POST /api/v1/reviews HTTP/1.1" 200 -
